In [1]:
import pandas as pd
import pyro
import pyro.distributions as dist
import torch
from itertools import product

In [15]:
employment_classes = ["employed", "unemployed"]

pairs =   pairs = [pair for pair in product(employment_classes, employment_classes) if pair[0] != pair[1]]

pairs_id = torch.arange(len(pairs))

fips_codes = [1001, 1005, 2001, 2003, 3005]

def fips_to_state(fips_codes):
    return [fips// 1000 for fips in fips_codes]

state_codes = list(set(fips_to_state(fips_codes)))


fips_id = torch.Tensor(pd.factorize(fips_codes)[0])
state_id = torch.Tensor(pd.factorize(state_codes)[0])


fips_observed = fips_id.repeat(4)
print("fo", fips_observed)
state_observed = fips_to_state(fips_observed)

mean = torch.tensor([.6]).repeat(20)
sd  = torch.tensor([.05]).repeat(20)
employment_observed = torch.normal(mean, sd)
unemployment_observed = 1 - employment_observed


fo tensor([0., 1., 2., 3., 4., 0., 1., 2., 3., 4., 0., 1., 2., 3., 4., 0., 1., 2.,
        3., 4.])


/tmp/ipykernel_7128/910625199.py:15: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  fips_id = torch.Tensor(pd.factorize(fips_codes)[0])
/tmp/ipykernel_7128/910625199.py:16: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  state_id = torch.Tensor(pd.factorize(state_codes)[0])


In [14]:
N_obs = len(employment_observed)
N_u = len(fips_id)
N_s = len(state_id)
N_pairs = len(pairs_id)

print("ns", N_obs, N_u, N_s, N_pairs)

def flux_sampling():
    observations_plate = pyro.plate("observations_plate", N_obs, dim=-1)
    counties_plate = pyro.plate("counties_plate", N_u, dim=-2)
    states_plate = pyro.plate("states_plate", N_s, dim=-3)
    flux_plate = pyro.plate("flux_plate", N_pairs, dim=-4)

    with flux_plate:
        federal_flux = pyro.sample("federal_flux", dist.Normal(0, 1)).squeeze()

        print("federal", federal_flux.shape)
        with states_plate:
            state_flux = pyro.sample("state_flux", dist.Normal(0, 1)).squeeze()

        print("state", state_flux.shape)

        with counties_plate:
            county_flux = pyro.sample("county_flux", dist.Normal(0, 1)).squeeze()
        print("county", county_flux.shape)

    with observations_plate:
        # making up some strange function
        employment_mean = pyro.deterministic("employment_mean", 
                                            federal_flux[0]  * federal_flux[1] + 
                                            state_flux[:,state_observed][0] * state_flux[:,state_observed][1] +
                                            county_flux[:,fips_observed][0] * county_flux[:state_observed][1])

        employment = pyro.sample("employment", dist.Normal(employment_mean, 0.05))

    return

with pyro.poutine.trace() as tr:
    flux_sampling()


for key in tr.trace.nodes.keys():
    print(key,tr.trace.nodes[key]["value"].shape)


ns 20 5 3 2
federal torch.Size([2])
state torch.Size([2, 3])
county torch.Size([2, 5])
ff0 tensor(-0.1671)


TypeError: only integer tensors of a single element can be converted to an index